In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch import nn, optim
import copy
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

/home/shlee/Desktop/MoE/MoE_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [2]:
model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model     = AutoModelForCausalLM.from_pretrained(
               model_id,
               torch_dtype=torch.float16,   # fp16 > 9기가 , 32는 14기가
               trust_remote_code=True
           ).to(DEVICE)

In [ ]:
# WIP model 구조를 봐서 어디가 MoE 인지 찾아야함. 아마 모델이 바뀌어도 어디가 MoE 인지 찾아서 for 문에 넣어주기만 하면 될 듯.
# 또, data 전처리하는 class 하나 만들어서 평준씨 data 에 적합하게 해야할듯.

# MoE param 찾아서 grad 킴. ('expert' or 'router')
tmp = False
for name, param in model.named_parameters():
    if 'mlp' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.down_proj.weight
model.layers.2.inp

In [ ]:
texts = ["반갑습니다 나는 이시현","좋아요."]
tokenizer.pad_token = tokenizer.eos_token # padding 을 사실상 안하겠다는 이야기지만, 대부분 이렇게 함.
inputs = tokenizer(texts, return_tensors="pt",
                   padding=True, truncation=True).to(DEVICE)
labels = inputs["input_ids"].clone().to(DEVICE)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

In [9]:
print(inputs)

{'input_ids': tensor([[128000, 101738, 115072,  39331, 109275,  23955,  30426, 102335],
        [128000, 125267, 112737,     13, 128001, 128001, 128001, 128001]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0]], device='cuda:0')}


In [10]:
model.train()
outputs = model(**inputs, labels=labels)
loss = outputs.loss
print(f"[Before Training] Loss: {loss.item():.4f}")

[Before Training] Loss: 7.9729


In [11]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [ ]:
print("\n[MoE Parameter Change Check]")
for name,p in model.named_parameters():
    if name == debug_name:
        debug_after = p.detach().clone() 
print(torch.all(debug_bef == debug_after))